# Библиотеки

In [14]:
import json

import pandas as pd
import numpy as np

# Предыстория 1

Привет! 

По проекту оптимизации экономии запасов сырья потребовалось сохранять в БД данные по химическом анализу жидкого чугуна. Сообщения поступают из кафки в формате json. 

Пример такого сообщения представлен в файле liquid_iron_msg.txt 

# Задача 1

Необходимо обработать сообщение и представить его в DataFrame таким образом, чтобы далее можно было отправлять его в таблицу для удобной последующей работы с данными. 

Примечание: поля RecordData, string, float, array, int служебные и указывают тип данных, их в базу отдельно записывтаь не нужно. Поле RecordData - структура (объект)

Атрибут 'op': 'U' также добавлять в бд не нужно, он указывает на обновление записи.

In [94]:
# Ваш код
with open('liquid_iron_msg.txt') as f:
    json_data = json.load(f)
    
keys = json_data.keys()
#print(json_data['ts'])
#print(json_data['op'])
#print(json_data['pk'].keys())

data = json_data['data']
data.keys()


df = pd.DataFrame()
    
index = 0
df.loc[index,'ts'] = json_data['ts']
for key in json_data['pk'].keys():
    df.loc[index, key] = json_data['pk'][key]

#Можно адаптировать под самый начальный лист, учитывая особенности вызова этой же функции внутри себя, но
#писалась конкретно под analyzes.chemical, чтобы разобрать лист и не делать код слишком нечитаемым
def list_parse(list_,index, column_name):
    iter_row = index
    if(str(type(list_)) == "<class 'list'>"):
        iter_row = index
        for list_of_dict in list_:
            if(str(type(list_of_dict)) == "<class 'dict'>"):
                
                for key_dict_value in list_of_dict.keys():
                    
                    if(str(type(list_of_dict[key_dict_value])) != "<class 'list'>"):
                        
                        if(str(type(list_of_dict[key_dict_value])) != "<class 'dict'>"):   
                            column = str(column_name) + '.' + str(key_dict_value)
                            df.loc[iter_row, column] = list_of_dict[key_dict_value]
                        else:
                            
                            if (len(list_of_dict[key_dict_value].keys()) == 1):
                                key = list(list_of_dict[key_dict_value].keys())[0]
                                column = str(column_name) + '.' + str(key_dict_value)
                                df.loc[iter_row, column] = list_of_dict[key_dict_value][key]
                    else:
                        #Пока в ней нет необходимости, но нужна обработка, если в листе снова лист, 
                        #Тогда нужно вызвать будет эту же функцию внутри себя.
                        1
                        
                iter_row+=1
    #одному analyzes.id соответствует целая серия хим.анализов, поэтому все поля продублированы на каждый анализ
    i=column_name.rfind('.')
    column_name_temp = column_name[:i]
    
    for column in list(df.columns):
        if((str(column).find(column_name_temp) != - 1)&(str(column).find(column_name) == -1)):
            df[column] = df[column].fillna(method='ffill')
            
    return iter_row


#Внешние ключи(столбцы)
for struct in data.keys():
#Cодержимое столбцов(если внутри будут еще словари, то название столбца = analyzes.id, analyzes.chemical.code и т.п)
    for column in data[struct].keys():
        value = data[struct][column]
        #Если в значениях словарь, то дальше разбираем, если нет то выполняем else(запись значения в столбец)
        if(str(type(value)) == "<class 'dict'>"):  
            #print(column,value)
            #input()
            #Символическая проверка для словарей с float,string и т.п, судя по структуре не должно быть больше одного ключа(либо внутри еще массив либо сразу значения)
            if(len(value.keys())==1):
                key = list(value.keys())[0]
                #Если внутри не лист, то пишем в столбец значения
                if(str(type(value[key])) != "<class 'list'>"):
                    column = str(struct) + '.' +str(column)
                    df.loc[index,column] = str(value[key])
                    
                else:
                    iter_row = index
                    #input(str(struct) +'.'+ str(column) +'\n keys'+str(list(value.keys())))
                    for list_of_dict in value[key]:
                        #print(type(list_of_dict))                      
                        for key_dict_value in list_of_dict.keys():
                            #input(str(struct) +'.'+ str(column) +'.'+str(key_dict_value)+'.'+str(list_of_dict[key_dict_value]))
                            column_name = str(struct)+'.'+str(column)+'.'+str(key_dict_value)
                            if(str(type(list_of_dict[key_dict_value])) == "<class 'dict'>"):
                                key_value = list(list_of_dict[key_dict_value].keys())[0]
                                #Если внутри не лист, то пишем в столбец значения
                                if(str(type(list_of_dict[key_dict_value][key_value])) != "<class 'list'>"):
                                    df.loc[iter_row,column_name] = str(list_of_dict[key_dict_value][key_value])
                                else:
                                    #Это для содержимого Chemical, т.к. там лист
                                    #input(list_parse(list_of_dict[key_dict_value][key_value],iter_row,column_name))
                                    iter_row = list_parse(list_of_dict[key_dict_value][key_value],iter_row,column_name)
                            else:
                                df.loc[iter_row,column_name] = str(list_of_dict[key_dict_value])
                        iter_row+=1
                        
                
            else:
            #Если внутри больше одного ключа, если вдруг будут переданы какие-то невалидные данные,
            #то как минимум это нужно увидеть на новых данных и понять закономерность
            #Выведем уведомление, что возможна потеря данных , где в словаре больше одного ключа
                try:
                    print('Необработанные данные внутри '+ str(struct) +'.'+ str(column) +'\n keys'+str(list(value.keys())))
                    print('продолжить? y/n')
                    if (input() == 'y'):
                        continue
                    else:
                        break
                except Exception as error:
                    print(error)
        else:
            column = str(struct) + '.' + str(column)
            df.loc[index,column]=value
    #Эта строчка для того, если бы было несколько объектов
    index +=1

#Функция для подсчета количества искомых подстрок в названии столбцов; понадобится при заполнении пустых строк ниже    
def len_of_all_indexes(origin_str, search_sub_str):
    list_ = []
    len_=len(origin_str)
    index = 0
    while (index < len_):
        i = origin_str.find(search_sub_str,index)
        if (i == -1):
            return len(list_)
        list_.append(index)
        index = i + 1
    return len(list_)
        
for column in list(df.columns):
    #Содержимое pk, и столбцы без вложенных полей(например RecordData.werks) сразу дублируем на все поля
    #analyzes уже продублированы еще в самой функции list_parse_;
    if ((len_of_all_indexes(str(column),'.') == 0)| (len_of_all_indexes(str(column),'.') == 1)):
        df[column] = df[column].fillna(method='ffill')
    
#В excel закинул для оценки возможности чтения в таком виде и дальнейшей записью в базу
#Ladles и Analyzes сами по себе не связаны и имеют разное количество строк, перемножать их не стал, 
#поэтому ladles заполнены толькосвоими 4 строками значений, далее все строки пустые
df.to_excel('export.xlsx',index=False)
df


,ts,systemCode,id,RecordData.werks,RecordData.werksName,RecordData.kceh,RecordData.kcehName,RecordData.unitCode,RecordData.unitName,RecordData.nplv,...,RecordData.analyzes.id,RecordData.analyzes.code,RecordData.analyzes.name,RecordData.analyzes.analysisDate,RecordData.analyzes.analyseDateSample,RecordData.analyzes.ladleNum,RecordData.analyzes.sampleNum,RecordData.analyzes.chemical.code,RecordData.analyzes.chemical.name,RecordData.analyzes.chemical.value
0,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811717,27,Хим. анализ ковшевого образца при наливе ковша,2021-08-03T16:15:12.000+03:00,None,65,None,9014.0,"Массовая доля кремния, Si",0.840000
1,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811717,27,Хим. анализ ковшевого образца при наливе ковша,2021-08-03T16:15:12.000+03:00,None,65,None,178.0,Содержание SiO2,NaN
2,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811717,27,Хим. анализ ковшевого образца при наливе ковша,2021-08-03T16:15:12.000+03:00,None,65,None,180.0,Содержание AL2O3,NaN
3,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811717,27,Хим. анализ ковшевого образца при наливе ковша,2021-08-03T16:15:12.000+03:00,None,65,None,179.0,Содержание MgO,NaN
4,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811717,27,Хим. анализ ковшевого образца при наливе ковша,2021-08-03T16:15:12.000+03:00,None,65,None,362.0,"Массовая доля оксида калия, K2O",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811715,23,Хим. анализ шлака,2021-08-03T16:13:06.000+03:00,None,None,1,182.0,Содержание FEO,0.322000
75,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811715,23,Хим. анализ шлака,2021-08-03T16:13:06.000+03:00,None,None,1,364.0,"Массовая доля оксида титана, TiO2",0.090000
76,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811715,23,Хим. анализ шлака,2021-08-03T16:13:06.000+03:00,None,None,1,181.0,Содержание MNO,0.104000
77,2021-08-03T16:32:41.316-03:00,10,161522908.0,1011,ПАО НЛМК,2,ДЦ-2,6.0,ДП-6,10423.0,...,6811715,23,Хим. анализ шлака,2021-08-03T16:13:06.000+03:00,None,None,1,177.0,Содержание CaO,42.959999


# Предыстория 2

В датасете dataset.csv собраны данные об датах оплат клиентом подписки на некие сервисы

Каждая оплата продлевает (добавляет к имеющемуся сроку) срок действия подписки:
	- для сервиса 1 на 30 дней
	- для сервиса 2 на 20 дней
	- для сервиса 3 на 40 дней

Ушедшим считается клиент у которого в течении 10 дней не действовала ни одна подписка на любой сервис

Датой ухода клиента считается дата окончания действия последней подписки перед периодом не действия ни одной подписки
Клиент может уходить несколько раз, возвращаясь


# Задача 2

Задача:
Написать на sql алгоритм позволяющий посчитать количество уходов для каждого месяца о котором есть данные.

In [6]:
import pyodbc

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-G99UC0T1\SQLEXPRESS;"
                      "Database=py_test;"
                      "Trusted_Connection=yes;")



In [6]:
query_create = "DROP TABLE IF EXISTS payments \
CREATE TABLE payments (\
  client_id VARCHAR(10) NOT NULL,\
  pay_dt DATE NOT NULL,\
  service_id INT NULL,\
);"
query_create

'DROP TABLE IF EXISTS payments CREATE TABLE payments (  client_id VARCHAR(10) NOT NULL,  pay_dt DATE NOT NULL,  service_id INT NULL,);'

In [7]:
cursor = cnxn.cursor()

cursor.execute(query_create)

cnxn.commit()
cursor.close()


In [10]:
# Ваш код
cursor = cnxn.cursor()
dset = pd.read_csv('dataset.csv',sep=';')
query_insert = "INSERT INTO payments (client_id,pay_dt,service_id) values(?,?,?)"
for index, row in dset.iterrows():
    cursor.execute(query_insert, row.client_id, row.pay_dt, row.service_id)
cnxn.commit()
cursor.close()

In [11]:
df = pd.read_sql('SELECT * FROM payments',cnxn)
df

,client_id,pay_dt,service_id
0,b,2010-08-15,1
1,a,2010-08-15,2
2,b,2010-08-15,3
3,b,2010-08-16,1
4,a,2010-08-16,2
...,...,...,...
67,a,2013-06-23,2
68,a,2013-07-27,1
69,c,2013-07-31,3
70,a,2013-09-07,1


In [9]:
#скрипт в папке sql (TEST_CASE.sql)
sql_query = 'USE py_test; DROP TABLE IF EXISTS #temp; SELECT ROW_NUMBER() OVER (PARTITION BY client_id ORDER BY pay_dt) AS row,CAST(NULL AS DATE) AS exit_date, CAST(NULL AS INT) AS balance, CAST(NULL AS DATE) AS global_until_date, * INTO #temp FROM payments; DROP TABLE IF EXISTS #clients SELECT row_number() OVER (ORDER BY client_id) as row, client_id INTO #clients FROM #temp GROUP BY client_id; DECLARE @max_client_iterator INT = (SELECT count(client_id) FROM #clients), @client_iterator INT = 1 ,@client_id VARCHAR(5), @cl_serv_iterator INT, @max_cl_serv_iterator INT, @service_id INT, @balance1 INT = 0, @until_date1 DATE, @pay_dt1 DATE, @balance2 INT = 0, @until_date2 DATE, @pay_dt2 DATE, @balance3 INT = 0, @until_date3 DATE, @pay_dt3 DATE, @service1_row INT, @service2_row INT, @service3_row INT, @global_next DATE, @@global_until_date DATE; WHILE (@client_iterator <= @max_client_iterator) BEGIN SET @client_id = (SELECT client_id FROM #clients WHERE row = @client_iterator); SET @cl_serv_iterator = 1; SET @max_cl_serv_iterator  = (SELECT count(row) FROM #temp WHERE client_id = @client_id); WHILE (@cl_serv_iterator <= @max_cl_serv_iterator) BEGIN SET @service_id =(SELECT service_id FROM #temp WHERE client_id = @client_id AND row = @cl_serv_iterator);	 SET @@global_until_date = (SELECT global_until_date FROM #temp WHERE client_id = @client_id AND row = @cl_serv_iterator); IF @service_id = 1 BEGIN										 SET @pay_dt1 = (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = @cl_serv_iterator); SET @balance1 = 0; SET @balance1 = 10 - DATEDIFF(DAY, (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = @service1_row), @pay_dt1) + (SELECT balance FROM #temp WHERE client_id = @client_id AND row = @service1_row); SET @service1_row = @cl_serv_iterator; IF @balance1 < 0 OR @balance1 IS NULL SET @balance1 = 0; UPDATE #temp SET balance = @balance1 WHERE client_id = @client_id AND row = @cl_serv_iterator; IF (@cl_serv_iterator = @max_cl_serv_iterator) SET @global_next = CAST(NULL AS DATE); ELSE SET @global_next = (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = (@cl_serv_iterator + 1)); SET @until_date1 = DATEADD(DAY, 10 + @balance1, @pay_dt1); PRINT (@until_date1); IF @until_date1 <= @@global_until_date SET @until_date1 = @@global_until_date IF DATEADD(DAY, 10, @until_date1) <= @global_next OR @cl_serv_iterator = @max_cl_serv_iterator UPDATE #temp SET exit_date = @until_date1 WHERE client_id = @client_id AND row = @cl_serv_iterator; IF @cl_serv_iterator != @max_cl_serv_iterator UPDATE #temp SET global_until_date = @until_date1 WHERE client_id = @client_id AND row = (@cl_serv_iterator +1); END IF @service_id = 2 BEGIN SET @pay_dt2 = (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = @cl_serv_iterator); SET @balance2 = 0; SET @balance2 = 20 - DATEDIFF(DAY, (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = @service2_row), @pay_dt2) + (SELECT balance FROM #temp WHERE client_id = @client_id AND row = @service2_row); SET @service2_row = @cl_serv_iterator; IF @balance2 < 0 OR @balance2 IS NULL SET @balance2 = 0; UPDATE #temp SET balance = @balance2 WHERE client_id = @client_id AND row = @cl_serv_iterator; IF (@cl_serv_iterator = @max_cl_serv_iterator) SET @global_next = CAST(NULL AS DATE); ELSE SET @global_next = (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = (@cl_serv_iterator + 1)); SET @until_date2 = DATEADD(DAY, 20 + @balance2, @pay_dt2); PRINT (@until_date2); IF @until_date2 <= @@global_until_date SET @until_date2 = @@global_until_date IF DATEADD(DAY, 10, @until_date2) <= @global_next OR @cl_serv_iterator = @max_cl_serv_iterator UPDATE #temp SET exit_date = @until_date2 WHERE client_id = @client_id AND row = @cl_serv_iterator; IF @cl_serv_iterator != @max_cl_serv_iterator UPDATE #temp SET global_until_date = @until_date2 WHERE client_id = @client_id AND row = (@cl_serv_iterator +1); END IF @service_id = 3 BEGIN SET @pay_dt3 = (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = @cl_serv_iterator); SET @balance3 = 0; SET @balance3 = 40 - DATEDIFF(DAY, (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = @service3_row), @pay_dt3) + (SELECT balance FROM #temp WHERE client_id = @client_id AND row = @service3_row); SET @service3_row = @cl_serv_iterator; IF @balance3 < 0 OR @balance3 IS NULL SET @balance3 = 0; UPDATE #temp SET balance = @balance3 WHERE client_id = @client_id AND row = @cl_serv_iterator; IF (@cl_serv_iterator = @max_cl_serv_iterator) SET @global_next = CAST(NULL AS DATE); ELSE SET @global_next = (SELECT pay_dt FROM #temp WHERE client_id = @client_id AND row = (@cl_serv_iterator + 1)); SET @until_date3 = DATEADD(DAY, 40 + @balance3, @pay_dt3); PRINT (@until_date3); IF @until_date3 <= @@global_until_date SET @until_date3 = @@global_until_date IF DATEADD(DAY, 10, @until_date3) <= @global_next OR @cl_serv_iterator = @max_cl_serv_iterator UPDATE #temp SET exit_date = @until_date3 WHERE client_id = @client_id AND row = @cl_serv_iterator; IF @cl_serv_iterator != @max_cl_serv_iterator UPDATE #temp SET global_until_date = @until_date3 WHERE client_id = @client_id AND row = (@cl_serv_iterator +1); END SET @cl_serv_iterator +=1; END SET @client_iterator +=1; END DROP TABLE IF EXISTS final; SELECT DISTINCT YEAR(exit_date) AS y ,MONTH(exit_date) AS m ,client_id INTO final FROM #temp WHERE exit_date IS NOT NULL; DROP TABLE IF EXISTS finish; SELECT DISTINCT y ,m ,Count(client_id) OVER (PARTITION BY y, m) AS exits_by_clients INTO finish FROM final'

In [10]:
sql_query

'USE py_test; DROP TABLE IF EXISTS #temp; SELECT ROW_NUMBER() OVER (PARTITION BY client_id ORDER BY pay_dt) AS row,CAST(NULL AS DATE) AS exit_date, CAST(NULL AS INT) AS balance, CAST(NULL AS DATE) AS global_until_date, * INTO #temp FROM payments; DROP TABLE IF EXISTS #clients SELECT row_number() OVER (ORDER BY client_id) as row, client_id INTO #clients FROM #temp GROUP BY client_id; DECLARE @max_client_iterator INT = (SELECT count(client_id) FROM #clients), @client_iterator INT = 1 ,@client_id VARCHAR(5), @cl_serv_iterator INT, @max_cl_serv_iterator INT, @service_id INT, @balance1 INT = 0, @until_date1 DATE, @pay_dt1 DATE, @balance2 INT = 0, @until_date2 DATE, @pay_dt2 DATE, @balance3 INT = 0, @until_date3 DATE, @pay_dt3 DATE, @service1_row INT, @service2_row INT, @service3_row INT, @global_next DATE, @@global_until_date DATE; WHILE (@client_iterator <= @max_client_iterator) BEGIN SET @client_id = (SELECT client_id FROM #clients WHERE row = @client_iterator); SET @cl_serv_iterator = 1;

In [11]:
cursor = cnxn.cursor()

cursor.execute(sql_query)

cnxn.commit()
cursor.close()

In [21]:
df = pd.read_sql('SELECT * FROM finish', cnxn)
print(df)

df1 = pd.read_sql('SELECT * FROM #temp', cnxn) #в этой таблице детальная информация по вылетам. В скрипте TEST_CASE.sql описано.
df1

       y   m  exits_by_clients
0   2010   9                 2
1   2010  12                 1
2   2011   1                 2
3   2011   2                 1
4   2011   3                 2
5   2011   4                 2
6   2011   5                 1
7   2011   8                 1
8   2011   9                 2
9   2011  11                 1
10  2011  12                 1
11  2012   1                 1
12  2012   2                 1
13  2012   4                 2
14  2012   6                 1
15  2012   7                 2
16  2012   8                 2
17  2012  10                 2
18  2012  12                 1
19  2013   2                 1
20  2013   3                 1
21  2013   5                 1
22  2013   6                 1
23  2013   7                 1
24  2013   8                 1
25  2013   9                 2
26  2013  12                 1


,row,exit_date,balance,global_until_date,client_id,pay_dt,service_id
0,1,None,0.0,None,a,2010-08-15,2
1,2,2010-09-24,19.0,2010-09-04,a,2010-08-16,2
2,3,None,0.0,2010-09-24,a,2010-12-24,1
3,4,None,0.0,2011-01-03,a,2010-12-24,2
4,5,None,0.0,2011-01-13,a,2010-12-24,3
...,...,...,...,...,...,...,...
67,8,None,NaN,None,e,2011-08-21,1
68,9,None,NaN,None,e,2011-09-17,1
69,10,None,NaN,None,e,2012-05-24,1
70,11,None,NaN,None,e,2012-06-25,3
